In [34]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer



In [35]:
# Load the Excel file
# file_path = 'Sample.xlsx'
# data = pd.read_excel(file_path)

file_path = 'sampled_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,QuestionId,AcceptedAnswerId,QuestionBody,QuestionTitle,QuestionTags,QuestionPostTypeId,QuestionOwnerUserId,QuestionScore,AnswerId,AnswerBody,...,AnswerTags,AnswerPostTypeId,AnswerOwnerUserId,AnswerScore,UserId,Reputation,UpVotes,DownVotes,Views,stratum_size
0,5356713,5363013,Let's say I've created a directory using modul...,MakeMaker: make test vs make dist,"['perl', 'cpan', 'makemaker']",1,280449.0,1,5363013,You can use make disttest which will create a ...,...,NaN,2,14660,5,14660,157630,2459,426,10204,1
1,4277388,4277427,Do I have to return the object and then put a ...,Increment an Integer within a HashMap,"['java', 'pointers', 'reference', 'hashmap']",1,106261.0,37,4277427,\nDo I have to return the object and then put ...,...,NaN,2,16883,14,16883,343927,3137,1318,32180,1
2,70652162,70652969,I'm currently having a problem that i'm unable...,Handling of rabbit messages via NESTJs microse...,"['mysql', 'typescript', 'rabbitmq', 'sequelize...",1,16400686.0,1,70652969,It looks to me like your Sequelize connection ...,...,NaN,2,205608,2,205608,106108,6166,519,9036,1
3,55766476,55766499,I have declared an int x as:\nint x = 1;\n\nLe...,can boolean value auto update itself if it was...,['java'],1,11364625.0,0,55766499,"There is no magic in Java, and variables do no...",...,NaN,2,522444,5,522444,284631,20109,92446,100067,1
4,930675,930765,"Sorry I don't quite get FP yet, I want to spli...",Functional paragraphs,"['haskell', 'functional-programming']",1,62721.0,4,930765,I'm only a beginning Haskell programmer (and t...,...,NaN,2,4958,4,4958,38666,2935,139,8980,1


In [36]:

# # Function to remove HTML tags from the text
# def clean_html(text):
#     return BeautifulSoup(text, "html.parser").get_text()

# # Apply the cleaning function to the QuestionBody
# data['CleanedQuestionBody'] = data['QuestionBody'].apply(clean_html)

# # Display the cleaned question bodies and the associated user IDs
# data[['CleanedQuestionBody', 'AnswerOwnerUserId']].head()


In [38]:
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\Mateo\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [39]:


def generate_embeddings(texts):
    embeddings = model.encode(texts)
    return embeddings

In [41]:
data['ada_embedding'] = data.QuestionBody.apply(lambda x: generate_embeddings(x))

In [42]:
data.to_csv('output/embedded_100k_reviews.csv', index=False)

In [43]:


def find_similar_questions(query_embedding, embeddings, top_k=10):
    similarities = cosine_similarity([query_embedding], embeddings)
    similar_indices = np.argsort(-similarities[0])[:top_k]
    return similar_indices


In [49]:

new_question = "in rust how do i declare a variable"
new_embedding = generate_embeddings(new_question)
similar_question_indices = find_similar_questions(new_embedding, np.array(data['ada_embedding'].tolist()))
similar_user_ids = data.iloc[similar_question_indices][['QuestionBody', 'AnswerOwnerUserId']]

In [50]:
similar_user_ids

,QuestionBody,AnswerOwnerUserId
75749,I installed Rust 1.13 and tried:\nfn main() {\...,493729
69514,I want to create an AttributeValue (a type def...,401059
70451,How do we use a variable multiple times as ref...,83839
93958,I'm learning Rust and the below code comes fro...,794457
62051,I am new to rust and am struggling with how t...,994153


In [51]:
similar_user_ids.iloc[1,0]

'I want to create an AttributeValue (a type defined by the rusoto_dynamodb library) in Rust, but I can\'t figure out how to do this from the documentation.\nHere\'s what I\'m trying:\nlet mut my_map = HashMap::new();\nmy_map.insert(String::from("key1"), AttributeValue::from("value1"));\n\nThis errors, pointing at value1 and saying:\nAttributeValue::from("value1"));\n                     ^^^^^^^^ expected struct `AttributeValue`, found `&str`\n\nI\'m fairly new to Rust and don\'t understand this error message, or how to read the docs. I think perhaps I want to use s - but how?\n'